<a href="https://colab.research.google.com/github/Talita1996/Compilador/blob/master/kmeans_pontos_ambiguos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importações

In [ ]:
pip install google-cloud-bigquery pandas-gbq

In [ ]:
import pandas as pd
from google.cloud import bigquery
import numpy as np

In [ ]:
project_id = 'eco-hangar-434017-p0'

query = 'SELECT * FROM `eco-hangar-434017-p0.dados.materiais` '

df_materiais = pd.read_gbq(query, project_id=project_id, dialect='standard')

df_materiais.head()

<ipython-input-3-eabbbdb78a92>:5: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_materiais = pd.read_gbq(query, project_id=project_id, dialect='standard')


,material,lote_sap,falta,dentro_da_politica,abaixo_min,acima_max,area_sobra,area_risco,area_falta,valor_monetario_anual,total_ocorrencias,categoria_un_monetaria,fator_mes,cluster
0,5279088,EX,0.33,0.0,0.67,0.00,0.000000,102.0000,43.0,323.968725,69,A,0.0,0.0
1,7606063,EX,0.18,0.0,0.74,0.09,22.000000,526.0000,23.0,4049.641338,68,A,0.0,0.0
2,37046,EX,0.26,0.0,0.28,0.47,108.000000,114.0000,33.0,4321.413672,58,A,0.0,0.0
3,1934015,EX,0.37,0.0,0.30,0.33,1065.000000,2193.0000,708.0,83.445398,83,A,0.0,0.0
4,3159340,EX,0.22,0.0,0.56,0.22,117.834503,12.0326,14.0,651.697219,45,A,0.0,0.0


In [ ]:
pip install kmodes

In [ ]:
import pickle

# Carregar o modelo salvo
with open('kprototypes_model.pkl', 'rb') as f:
    kproto = pickle.load(f)

# Análise das distâncias

In [ ]:
data = df_materiais.drop(['cluster', 'material'], axis=1).copy()

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np

categorical_columns = ['lote_sap', 'categoria_un_monetaria']
numerical_columns = ['falta', 'dentro_da_politica', 'abaixo_min', 'acima_max', 'area_sobra', 'area_risco', 'area_falta', 'valor_monetario_anual', 'total_ocorrencias', 'fator_mes']

# Normalizar os dados numéricos
scaler = StandardScaler()
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

data.head()

,lote_sap,falta,dentro_da_politica,abaixo_min,acima_max,area_sobra,area_risco,area_falta,valor_monetario_anual,total_ocorrencias,categoria_un_monetaria,fator_mes
0,EX,1.059202,-0.863885,4.137389,-1.293581,-0.006733,-0.006924,-0.004573,-0.040518,0.440501,A,-0.883754
1,EX,0.407043,-0.863885,4.605927,-1.079377,-0.006732,-0.005734,-0.004667,-0.032631,0.414541,A,-0.883754
2,EX,0.754861,-0.863885,1.526964,-0.174958,-0.006727,-0.006891,-0.004620,-0.032056,0.154941,A,-0.883754
3,EX,1.233111,-0.863885,1.660832,-0.508165,-0.006679,-0.001053,-0.001475,-0.041027,0.803941,A,-0.883754
4,EX,0.580952,-0.863885,3.401115,-0.769970,-0.006727,-0.007177,-0.004709,-0.039824,-0.182539,A,-0.883754


In [ ]:
from scipy.spatial.distance import cdist

categorical_columns = [0, 10]
numerical_columns = [i for i in range(data.shape[1]) if i not in categorical_columns]

X_categorical = data.iloc[:, categorical_columns].values
X_numerical = data.iloc[:, numerical_columns].values

print("Variáveis categóricas:", X_categorical)
print("Variáveis numéricas:", X_numerical)

Variáveis categóricas: [['EX' 'A']
 ['EX' 'A']
 ['EX' 'A']
 ...
 ['ZB' 'A+']
 ['ZC' 'A+']
 ['ZQ' 'A+']]
Variáveis numéricas: [[ 1.05920216 -0.86388476  4.1373894  ... -0.04051824  0.44050094
  -0.88375373]
 [ 0.40704334 -0.86388476  4.60592734 ... -0.03263149  0.41454095
  -0.88375373]
 [ 0.75486138 -0.86388476  1.5269637  ... -0.03205618  0.15494109
  -0.88375373]
 ...
 [ 0.23313433 -0.34848885  1.5269637  ...  0.035307    1.27122051
  -0.61872114]
 [ 0.23313433 -0.83675866 -0.34718807 ... -0.01310982  1.24526052
  -0.35368856]
 [ 0.23313433 -0.29423665  1.25922774 ...  0.2468672  -0.59789852
  -0.68497929]]


In [ ]:
centroids_numerical = []
centroids_categorical = []

for centroid in kproto.cluster_centroids_:
  centroids_categorical.append([centroid[i] for i in [10, 11]])
  centroids_numerical.append([centroid[i] for i in range(0,10)])

centroids_numerical_converted = []
for centroid in centroids_numerical:
    centroids_numerical_converted.append([float(coord) for coord in centroid])  # Convert each coordinate to float

centroids_numerical = centroids_numerical_converted

print("Centroides Categóricos:", centroids_categorical)
print("Centroides Numéricos:", centroids_numerical)

Centroides Categóricos: [['Z3', 'A'], ['EX', 'D'], ['EX', 'D'], ['Z1', 'A+'], ['EX', 'A']]
Centroides Numéricos: [[-0.05698881453900826, -0.27291255083152655, 3.408886897927374, -0.9402090984390173, 0.049862976147553854, 0.1000132873516483, 0.00021176404969343828, 0.030957019787325797, -0.18971174319639558, -0.07979990667856521], [-0.31201501197411396, -0.6701399986509056, -0.2404474117069868, 0.8441733079134687, -0.0006246032261706913, -0.0067254081911728904, -0.0040564526363862565, -0.014784241916326043, 0.31621745147897057, -0.08157210641051316], [3.0803600854189868, -0.45003433488846906, -0.24516278208537773, -1.2042634810263522, -0.006697756956865097, -0.003344650029953347, 0.001173141232286192, 0.18770768658359832, -0.8244531932014827, -0.5838171120104094], [0.45052059862958904, -0.8367586635890533, 0.9245577751020363, 0.1582488612482863, -0.006708928851103639, -0.0011135872305531045, 277.646320830228, -0.01340589967288764, 1.089500601663811, -0.6187211435933653], [-0.24641250681

In [ ]:
# 1. Calcular a distância euclidiana para as colunas numéricas
distances_numerical = cdist(X_numerical, centroids_numerical, metric='euclidean')

# 2. Calcular a distância de Hamming para as colunas categóricas
def hamming_distance(a, b):
    return np.sum(a != b, axis=1)

distances_categorical = np.zeros((X_categorical.shape[0], len(centroids_categorical)))

for i, centroid in enumerate(centroids_categorical):
    distances_categorical[:, i] = hamming_distance(X_categorical, centroid)

# 3. Combine as distâncias usando o fator gamma
gamma = kproto.gamma
combined_distances = distances_numerical + gamma * distances_categorical

# Identifique os clusters mais próximos e os segundos mais próximos
closest_clusters = np.argsort(combined_distances, axis=1)

# Distância ao cluster atribuído e ao segundo mais próximo
first_closest = combined_distances[np.arange(combined_distances.shape[0]), closest_clusters[:, 0]]
second_closest = combined_distances[np.arange(combined_distances.shape[0]), closest_clusters[:, 1]]

# Defina um limiar para ambiguidade
threshold = 0.1
ambiguous_points = np.where(second_closest - first_closest < threshold)[0]

print("Pontos ambíguos:", ambiguous_points)

Pontos ambíguos: [    2    37    42 ... 77944 78020 78022]


In [ ]:
ambiguous_df = df_materiais.iloc[ambiguous_points]

ambiguous_df.head()

,material,lote_sap,falta,dentro_da_politica,abaixo_min,acima_max,area_sobra,area_risco,area_falta,valor_monetario_anual,total_ocorrencias,categoria_un_monetaria,fator_mes,cluster
2,37046,EX,0.26,0.00,0.28,0.47,108.000,114.0000,33.000000,4321.413672,58,A,0.0,0.0
37,6313479,EX,0.11,0.55,0.34,0.00,0.000,28.0000,18.000000,1795.693183,82,A,0.0,0.0
42,6051555,EX,0.39,0.26,0.29,0.06,2.807,2.0291,73.664999,404.635891,31,A,0.0,0.0
45,5963662,EX,0.58,0.03,0.37,0.02,1.000,50.0000,40.000000,120.681016,62,A,0.0,0.0
80,9563481,EX,0.20,0.20,0.28,0.33,75.000,29.0000,59.000000,1165.746802,101,A,0.0,0.0


In [ ]:
ambiguous_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1547 entries, 2 to 78022
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   material                1547 non-null   Int64  
 1   lote_sap                1547 non-null   object 
 2   falta                   1547 non-null   float64
 3   dentro_da_politica      1547 non-null   float64
 4   abaixo_min              1547 non-null   float64
 5   acima_max               1547 non-null   float64
 6   area_sobra              1547 non-null   float64
 7   area_risco              1547 non-null   float64
 8   area_falta              1547 non-null   float64
 9   valor_monetario_anual   1547 non-null   float64
 10  total_ocorrencias       1547 non-null   Int64  
 11  categoria_un_monetaria  1547 non-null   object 
 12  fator_mes               1547 non-null   float64
 13  cluster                 1547 non-null   float64
dtypes: Int64(2), float64(10), object(2)
memory u

In [ ]:
ambiguous_df['cluster'].value_counts()

,count
cluster,
1.0,932
4.0,391
0.0,122
2.0,102


## Visualização centróides

In [86]:
centroids_reescaled = []

for centroid in centroids_numerical:
    temp = np.array(centroid, dtype=float).reshape(1, -1)
    centroids_reescaled.append(scaler.inverse_transform(temp)[0])

# Converte a lista de listas em um array numpy de coordenadas
centroids_reescaled = np.array(centroids_reescaled)

print(centroids_reescaled)

[[7.32701321e-02 2.17861099e-01 5.61161057e-01 1.48472518e-01
  1.12719307e+06 3.81858946e+04 1.07001163e+03 3.40886216e+04
  4.47236898e+01 3.03341500e+00]
 [1.46127485e-02 7.14237432e-02 1.59471532e-02 8.98197632e-01
  1.21654126e+05 1.72879510e+02 1.53953986e+02 1.24806160e+04
  6.42124979e+01 3.02672827e+00]
 [7.94877088e-01 1.52565396e-01 1.52426725e-02 3.75275764e-02
  6.97791114e+02 1.37687409e+03 1.27634532e+03 1.08137066e+05
  2.02729278e+01 1.13169713e+00]
 [1.90000000e-01 1.00000000e-02 1.90000000e-01 6.10000000e-01
  4.75285403e+02 2.17142590e+03 5.95903222e+07 1.31317398e+04
  9.40000000e+01 1.00000000e+00]
 [2.97016740e-02 8.03986057e-01 2.59948928e-02 1.40528150e-01
  7.96129506e+02 2.05289245e+02 2.88931978e+01 5.85137083e+03
  4.06989178e+01 4.48635848e+00]]


# Visualização dos clusters

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
df_cluster_0 = df_materiais[df_materiais['cluster'] == 0]
df_cluster_1 = df_materiais[df_materiais['cluster'] == 1]
df_cluster_2 = df_materiais[df_materiais['cluster'] == 2]
df_cluster_3 = df_materiais[df_materiais['cluster'] == 3]
df_cluster_4 = df_materiais[df_materiais['cluster'] == 4]

In [ ]:
df_cluster_0_ambiguous = ambiguous_df[ambiguous_df['cluster'] == 0]
df_cluster_1_ambiguous = ambiguous_df[ambiguous_df['cluster'] == 1]
df_cluster_2_ambiguous = ambiguous_df[ambiguous_df['cluster'] == 2]
df_cluster_3_ambiguous = ambiguous_df[ambiguous_df['cluster'] == 3]
df_cluster_4_ambiguous = ambiguous_df[ambiguous_df['cluster'] == 4]

## Cluster 0: abaixo_min

In [87]:
df_cluster_0.describe()

,material,falta,dentro_da_politica,abaixo_min,acima_max,area_sobra,area_risco,area_falta,valor_monetario_anual,total_ocorrencias,fator_mes,cluster
count,4694.0,4694.000000,4694.000000,4694.000000,4694.000000,4.694000e+03,4.694000e+03,4.694000e+03,4.694000e+03,4694.0,4694.000000,4694.0
mean,4794297.874947,0.073270,0.217861,0.561161,0.148473,1.127193e+06,3.818589e+04,1.070012e+03,3.408862e+04,44.72369,3.033415,0.0
std,2898472.364498,0.129278,0.191247,0.224150,0.177304,7.704228e+07,1.447499e+06,3.811594e+04,1.492471e+05,31.587349,2.926553,0.0
min,470.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,0.000000,0.0
25%,2114815.25,0.000000,0.030000,0.380000,0.000000,0.000000e+00,4.600000e+01,0.000000e+00,6.753365e+02,15.0,1.000000,0.0
50%,4880495.5,0.000000,0.200000,0.500000,0.070000,1.000000e+01,2.490000e+02,0.000000e+00,3.933520e+03,40.0,2.000000,0.0
75%,7272621.5,0.100000,0.350000,0.700000,0.270000,2.567500e+02,1.164000e+03,1.600000e+01,1.404987e+04,65.0,4.000000,0.0
max,9970153.0,0.630000,0.680000,1.000000,1.000000,5.278381e+09,8.804939e+07,2.165090e+06,2.674884e+06,101.0,12.000000,0.0


In [93]:
'''ordem do centroide
falta	dentro_da_politica abaixo_min	acima_max	area_sobra	area_risco	area_falta	valor_monetario_anual	total_ocorrencias	fator_mes lote_sap categoria_un_monetaria'''

# cluster
fig = px.scatter(df_cluster_0, x='dentro_da_politica', y='abaixo_min', color='cluster',
                 hover_data=['material', 'total_ocorrencias', 'categoria_un_monetaria', 'lote_sap', 'falta'])

# pontos ambíguos
fig.add_trace(go.Scatter(x=df_cluster_0_ambiguous['dentro_da_politica'],
                         y=df_cluster_0_ambiguous['abaixo_min'],
                         mode='markers',
                         marker=dict(color='black', size=10),
                         name='Ambiguous Points',
                         hovertext=df_cluster_0_ambiguous[['material', 'total_ocorrencias', 'categoria_un_monetaria', 'lote_sap', 'falta']].astype(str).apply('<br>'.join, axis=1)))

# centróide
fig.add_trace(go.Scatter(x=[centroids_reescaled[0][1]],
                         y=[centroids_reescaled[0][2]],  # Coordenada do centróide no cluster 0
                         mode='markers',
                         marker=dict(color='blue', size=15, symbol='x'),  # Azul e símbolo 'x' para o centróide
                         name='Cluster 0 Centroid'))

fig.show()


## Cluster 1: acima_max

In [89]:
df_cluster_1.describe()

,material,falta,dentro_da_politica,abaixo_min,acima_max,area_sobra,area_risco,area_falta,valor_monetario_anual,total_ocorrencias,fator_mes,cluster
count,42311.0,42311.000000,42311.000000,42311.000000,42311.000000,4.231100e+04,42311.000000,4.231100e+04,4.231100e+04,42311.0,42311.000000,42311.0
mean,4879333.509844,0.014613,0.071424,0.015947,0.898198,1.216541e+05,172.879510,1.539540e+02,1.248062e+04,64.212498,3.026728,1.0
std,2881324.041721,0.053358,0.119452,0.047406,0.157564,8.544192e+06,7357.654693,2.310205e+04,1.360790e+05,37.347728,3.557048,0.0
min,42.0,0.000000,0.000000,0.000000,0.210000,2.212524e-07,0.000000,0.000000e+00,0.000000e+00,1.0,0.000000,1.0
25%,2151163.0,0.000000,0.000000,0.000000,0.820000,1.070000e+02,0.000000,0.000000e+00,0.000000e+00,24.0,0.250000,1.0
50%,4798556.0,0.000000,0.000000,0.000000,1.000000,4.900000e+02,0.000000,0.000000e+00,2.061360e+01,76.0,2.000000,1.0
75%,7419420.0,0.000000,0.100000,0.000000,1.000000,2.045000e+03,0.000000,0.000000e+00,1.981739e+03,101.0,3.000000,1.0
max,9999570.0,0.510000,0.590000,0.360000,1.000000,1.573743e+09,911908.000000,4.740448e+06,1.454189e+07,101.0,12.000000,1.0


In [94]:
'''ordem do centroide
falta	dentro_da_politica abaixo_min	acima_max	area_sobra	area_risco	area_falta	valor_monetario_anual	total_ocorrencias	fator_mes lote_sap categoria_un_monetaria'''

# cluster
fig = px.scatter(df_cluster_1, x='dentro_da_politica', y='acima_max', color='cluster',
                 hover_data=['material', 'total_ocorrencias', 'categoria_un_monetaria', 'lote_sap', 'falta'])

# pontos ambíguos
fig.add_trace(go.Scatter(x=df_cluster_1_ambiguous['dentro_da_politica'],
                         y=df_cluster_1_ambiguous['acima_max'],
                         mode='markers',
                         marker=dict(color='black', size=10),
                         name='Ambiguous Points',
                         hovertext=df_cluster_1_ambiguous[['material', 'total_ocorrencias', 'categoria_un_monetaria', 'lote_sap', 'falta']].astype(str).apply('<br>'.join, axis=1)))

# centróide
fig.add_trace(go.Scatter(x=[centroids_reescaled[1][1]],
                         y=[centroids_reescaled[1][3]],
                         mode='markers',
                         marker=dict(color='blue', size=15, symbol='x'),
                         name='Cluster 1 Centroid'))

fig.show()


## Cluster 2: falta

In [95]:
df_cluster_2.describe()

,material,falta,dentro_da_politica,abaixo_min,acima_max,area_sobra,area_risco,area_falta,valor_monetario_anual,total_ocorrencias,fator_mes,cluster
count,6346.0,6346.000000,6346.000000,6346.000000,6346.000000,6.346000e+03,6.346000e+03,6.346000e+03,6.346000e+03,6346.0,6346.000000,6346.0
mean,5226165.372361,0.794877,0.152565,0.015243,0.037528,6.977911e+02,1.376874e+03,1.276345e+03,1.081371e+05,20.272928,1.131697,2.0
std,3079378.25094,0.214213,0.189322,0.053637,0.103588,4.707406e+04,9.091142e+04,4.630780e+04,1.595929e+06,27.414937,1.760075,0.0
min,20.0,0.260000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,1.000000e-03,0.000000e+00,1.0,0.000000,2.0
25%,2175898.75,0.600000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,2.0,0.000000,2.0
50%,5507621.5,0.860000,0.040000,0.000000,0.000000,0.000000e+00,0.000000e+00,1.000000e+01,0.000000e+00,8.0,0.250000,2.0
75%,8000851.25,1.000000,0.300000,0.000000,0.000000,0.000000e+00,0.000000e+00,4.000000e+01,4.115907e+01,26.0,2.000000,2.0
max,9999170.0,1.000000,0.630000,0.400000,0.630000,3.741676e+06,7.218054e+06,3.171238e+06,9.730213e+07,101.0,12.000000,2.0


In [100]:
'''ordem do centroide
falta	dentro_da_politica abaixo_min	acima_max	area_sobra	area_risco	area_falta	valor_monetario_anual	total_ocorrencias	fator_mes lote_sap categoria_un_monetaria'''

# cluster
fig = px.scatter(df_cluster_2, x='dentro_da_politica', y='falta', color='cluster',
                 hover_data=['material', 'total_ocorrencias', 'categoria_un_monetaria', 'lote_sap', 'falta'])

# pontos ambíguos
fig.add_trace(go.Scatter(x=df_cluster_2_ambiguous['dentro_da_politica'],
                         y=df_cluster_2_ambiguous['falta'],
                         mode='markers',
                         marker=dict(color='black', size=10),
                         name='Ambiguous Points',
                         hovertext=df_cluster_2_ambiguous[['material', 'total_ocorrencias', 'categoria_un_monetaria', 'lote_sap', 'falta']].astype(str).apply('<br>'.join, axis=1)))

# centróide
fig.add_trace(go.Scatter(x=[centroids_reescaled[2][1]],
                         y=[centroids_reescaled[2][0]],
                         mode='markers',
                         marker=dict(color='blue', size=15, symbol='x'),
                         name='Cluster 2 Centroid'))

fig.show()


## Cluster 4: dentro_da_politica

In [97]:
df_cluster_4.describe()

,material,falta,dentro_da_politica,abaixo_min,acima_max,area_sobra,area_risco,area_falta,valor_monetario_anual,total_ocorrencias,fator_mes,cluster
count,24671.0,24671.000000,24671.000000,24671.000000,24671.000000,2.467100e+04,2.467100e+04,24671.000000,2.467100e+04,24671.0,24671.000000,24671.0
mean,5060259.404078,0.029702,0.803986,0.025995,0.140528,7.961295e+02,2.052892e+02,28.893198,5.851371e+03,40.698918,4.486358,4.0
std,3014134.986328,0.079028,0.199360,0.062128,0.182445,1.198701e+04,7.892281e+03,1112.737486,1.112438e+05,35.566967,4.259632,0.0
min,41.0,0.000000,0.130000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,1.0,0.000000,4.0
25%,2151319.5,0.000000,0.640000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,10.0,0.250000,4.0
50%,5002028.0,0.000000,0.860000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,3.299732e+02,36.0,3.000000,4.0
75%,8014737.5,0.000000,1.000000,0.000000,0.280000,1.030000e+02,0.000000e+00,0.000000,1.819418e+03,62.0,8.000000,4.0
max,9998110.0,0.530000,1.000000,0.350000,0.700000,1.087168e+06,1.131420e+06,130476.528870,1.043499e+07,101.0,12.000000,4.0


In [99]:
'''ordem do centroide
falta	dentro_da_politica abaixo_min	acima_max	area_sobra	area_risco	area_falta	valor_monetario_anual	total_ocorrencias	fator_mes lote_sap categoria_un_monetaria'''

# cluster
fig = px.scatter(df_cluster_4, x='dentro_da_politica', y='acima_max', color='cluster',
                 hover_data=['material', 'total_ocorrencias', 'categoria_un_monetaria', 'lote_sap', 'falta'])

# pontos ambíguos
fig.add_trace(go.Scatter(x=df_cluster_4_ambiguous['dentro_da_politica'],
                         y=df_cluster_4_ambiguous['acima_max'],
                         mode='markers',
                         marker=dict(color='black', size=10),
                         name='Ambiguous Points',
                         hovertext=df_cluster_4_ambiguous[['material', 'total_ocorrencias', 'categoria_un_monetaria', 'lote_sap', 'falta']].astype(str).apply('<br>'.join, axis=1)))

# centróide
fig.add_trace(go.Scatter(x=[centroids_reescaled[4][1]],
                         y=[centroids_reescaled[4][3]],
                         mode='markers',
                         marker=dict(color='blue', size=15, symbol='x'),
                         name='Cluster 2 Centroid'))

fig.show()
